In [1]:
import torch
from flashdiv.flows.egnn import EGNN_dynamics
from flashdiv.flows.eqtf_old import EqTransformerFlowLJ
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = EqTransformerFlowLJ(input_dim=2).to(device)
# gnn = EGNN_dynamics().to(device)
eqtf = EqTransformerFlowLJ(input_dim=2).to(device)

In [24]:
x = torch.randn(256, 9, 2).to(device)
t = torch.rand(256).to(device)
#timeit
%timeit eqtf.forward(x, t)
%timeit eqtf.forward_fast(x, t)


136 ms ± 419 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1 ms ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
out1 = eqtf.forward(x, t)
out2 = eqtf.forward_fast(x, t)
diff = torch.norm(out1 - out2)
print(f"Difference between forward and forward_fast: {diff.item()}")

Difference between forward and forward_fast: 0.0


In [22]:
def test_equivariance(model):
    import math

    device = next(model.parameters()).device
    torch.manual_seed(0)  # For reproducibility

    n_particles = 3
    n_batch = 1

    # Generate random positions for 9 particles in 2D
    xs = torch.randn(n_batch, n_particles, 2, device=device)
    t = torch.tensor([0.1], device=device)

    # --- Rotation Equivariance Test ---
    theta = math.pi / 3  # 60 degrees rotation
    R = torch.tensor([[math.cos(theta), -math.sin(theta)],
                      [math.sin(theta),  math.cos(theta)]], dtype=torch.float32, device=device)
    xs_rot = torch.matmul(xs, R.T)

    vel = model.forward(xs, t)
    vel_rot = model.forward(xs_rot, t)
    vel_rot_expected = torch.matmul(vel, R.T)

    err_rotation = (vel_rot - vel_rot_expected).abs().max().item()
    print(f"[Rotation equivariance error]: {err_rotation:.6e}")

    # --- Permutation Equivariance Test ---
    perm = torch.randperm(n_particles, device=device)
    xs_perm = xs[:, perm, :]
    vel_perm = model.forward(xs_perm, t)
    vel_expected = vel[:, perm, :]

    err_perm = (vel_perm - vel_expected).abs().max().item()
    print(f"[Permutation equivariance error]: {err_perm:.6e}")

    # --- Center of Mass Preservation Test ---
    com_input = xs.mean(dim=1)  # should be (0, 0)
    com_output = vel.mean(dim=1)  # check output COM
    err_com = com_output.norm(dim=-1).max().item()
    print(f"[Center of mass error]: {err_com:.6e}")


In [23]:
test_equivariance(eqtf)


[Rotation equivariance error]: 2.384186e-07
[Permutation equivariance error]: 0.000000e+00
[Center of mass error]: 7.942639e-02
